# 回帰分析による因果推論
テレビCMの広告効果の推定を例に、回帰分析による因果推論を実装します。


In [1]:
import numpy as np
from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

$x_1\in[15,76]$ : 年齢

$x_2\in\{0,1\}$ : 性別

$
y=-x_1+30x_2+10z+80+10\epsilon_y
$

$
\mathrm{Prob}(Z)=\mathrm{sigmoid}(0.1(x_1+10(1-x_2)-40+5\epsilon_z))
$

In [2]:
data = np.loadtxt("data/cm_purchase.csv",delimiter=",")

回帰分析を実施

In [3]:
X=data[:,0:3]
Y=data[:,3]
reg = LinearRegression().fit(X, Y)
print("係数：", reg.coef_)

係数： [-1.00808243 32.35465652 10.97612368]


- 逆確率重み付け法（IPTW）による因果推論

In [4]:
X = data[:,0:2]
Z = data[:,2]
reg = LogisticRegression().fit(X,Z)
print("係数beta：", reg.coef_)
print("係数alpha：", reg.intercept_)

係数beta： [[ 0.08716475 -0.54525891]]
係数alpha： [-2.99898819]


傾向スコア

In [5]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])
print("----")
print(Z[0:5])

[[0.48705836 0.51294164]
 [0.6002042  0.3997958 ]
 [0.77828382 0.22171618]
 [0.86854066 0.13145934]
 [0.08448165 0.91551835]]
----
[1. 0. 0. 0. 1.]


平均処置効果(ATE)

In [6]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:, 0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)

推定したATE 15.393159778678676


- Doubly Robust法（DR法）による因果推論

In [7]:
X = data[:,0:3]
Y = data[:,3]
reg2 = LinearRegression().fit(X, Y)

# Z=0の場合
X_0 = X.copy()
X_0[2] = 0
Y_0 = reg2.predict(X_0)

# Z=1の場合
X_1 = X.copy()
X_1[2] = 1
Y_1 = reg2.predict(X_1)

In [8]:
# 説明変数
X = data[:,0:2]
# 被説明変数（目的変数）
Z = data[:,2]
# 回帰の実施
reg = LogisticRegression().fit(X, Z)

# 傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる

[[0.48705836 0.51294164]
 [0.6002042  0.3997958 ]
 [0.77828382 0.22171618]
 [0.86854066 0.13145934]
 [0.08448165 0.91551835]]


In [9]:
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()
print("推定したATE", ATE)

推定したATE 0.4890801942990295
